In [1]:
input_rdd = sc.textFile('/FileStore/tables/num_new_100.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)

In [2]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.05
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/num_new_100.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import sys
numRecords：100.0
frequency threshold：5.0
FreqSet-2:
seconds process time: 0.073048
seconds wall time: 1.4607875347137451
[(frozenset({'39', '48'}), 0.01), (frozenset({'225', '32'}), 0.0011), (frozenset({'39', '225'}), 0.0011), (frozenset({'32', '48'}), 0.01), (frozenset({'38', '225'}), 0.0011), (frozenset({'38', '48'}), 0.01), (frozenset({'32', '1146'}), 0.0007000000000000001), (frozenset({'38', '1146'}), 0.0007000000000000001), (frozenset({'39', '237'}), 0.0007000000000000001), (frozenset({'38', '237'}), 0.0007000000000000001), (frozenset({'237', '32'}), 0.0007000000000000001), (frozenset({'39', '1146'}), 0.0007000000000000001), (frozenset({'39', '1715'}), 0.0006), (frozenset({'38', '1715'}), 0.0006), (frozenset({'32', '1715'}), 0.0006), (frozenset({'37', '41'}), 0.0007000000000000001), (frozenset({'48', '89'}), 0.0008), (frozenset({'38', '170'}), 0.0018), (frozenset({'39', '170'}), 0.0018), (frozenset({'170', '32'}), 0.0018), (frozenset({'310', '41'}), 0.001), (frozenset({'475', '41'}), 0.0008), (frozenset({'790', '41'}), 0.0009), (frozenset({'38', '110'}), 0.0021), (frozenset({'39', '110'}), 0.0021), (frozenset({'110', '32'}), 0.0021), (frozenset({'105', '48'}), 0.0006), (frozenset({'38', '3904'}), 0.0006), (frozenset({'3904', '32'}), 0.0006), (frozenset({'39', '3904'}), 0.0006), (frozenset({'475', '48'}), 0.0008), (frozenset({'32', '604'}), 0.0006), (frozenset({'39', '604'}), 0.0006), (frozenset({'38', '604'}), 0.0006), (frozenset({'310', '48'}), 0.001), (frozenset({'105', '41'}), 0.0006), (frozenset({'790', '48'}), 0.0009), (frozenset({'48', '604'}), 0.0006), (frozenset({'170', '41'}), 0.0018), (frozenset({'110', '41'}), 0.0021), (frozenset({'310', '38'}), 0.001), (frozenset({'310', '32'}), 0.001), (frozenset({'310', '39'}), 0.001), (frozenset({'790', '32'}), 0.0009), (frozenset({'790', '38'}), 0.0009), (frozenset({'790', '39'}), 0.0009), (frozenset({'3904', '41'}), 0.0006), (frozenset({'475', '32'}), 0.0008), (frozenset({'475', '38'}), 0.0008), (frozenset({'475', '39'}), 0.0008), (frozenset({'170', '48'}), 0.0018), (frozenset({'39', '105'}), 0.0006), (frozenset({'105', '32'}), 0.0006), (frozenset({'38', '105'}), 0.0006), (frozenset({'604', '41'}), 0.0006), (frozenset({'110', '48'}), 0.0021), (frozenset({'3904', '48'}), 0.0006), (frozenset({'38', '32'}), 0.01), (frozenset({'38', '39'}), 0.01), (frozenset({'39', '32'}), 0.01), (frozenset({'225', '48'}), 0.0011), (frozenset({'147', '41'}), 0.0006), (frozenset({'36', '41'}), 0.0021), (frozenset({'237', '48'}), 0.0007000000000000001), (frozenset({'32', '89'}), 0.0008), (frozenset({'38', '89'}), 0.0008), (frozenset({'39', '89'}), 0.0008), (frozenset({'1146', '48'}), 0.0007000000000000001), (frozenset({'48', '1715'}), 0.0006), (frozenset({'48', '41'}), 0.01), (frozenset({'237', '41'}), 0.0007000000000000001), (frozenset({'37', '32'}), 0.0007000000000000001), (frozenset({'37', '39'}), 0.0007000000000000001), (frozenset({'37', '38'}), 0.0007000000000000001), (frozenset({'225', '41'}), 0.0011), (frozenset({'1715', '41'}), 0.0006), (frozenset({'147', '48'}), 0.0006), (frozenset({'36', '48'}), 0.0021), (frozenset({'1146', '41'}), 0.0007000000000000001), (frozenset({'36', '225'}), 0.0005), (frozenset({'32', '41'}), 0.01), (frozenset({'38', '41'}), 0.01), (frozenset({'39', '41'}), 0.01), (frozenset({'37', '48'}), 0.0007000000000000001), (frozenset({'38', '36'}), 0.0021), (frozenset({'39', '36'}), 0.0021), (frozenset({'36', '32'}), 0.0021), (frozenset({'147', '32'}), 0.0006), (frozenset({'38', '147'}), 0.0006), (frozenset({'39', '147'}), 0.0006), (frozenset({'89', '41'}), 0.0008)]

FreqSet-3:
seconds process time: 0.08019199999999982
seconds wall time: 1.142749309539795
[(frozenset({'110', '48', '41'}), 0.0021), (frozenset({'38', '32', '604'}), 0.0006), (frozenset({'39', '

In [3]:
input_rdd = sc.textFile('/FileStore/tables/num_500.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)

In [4]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.01
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/num_500.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import sys
FreqSet-2:
seconds process time: 0.11290500000000048
seconds wall time: 1.0477182865142822
[(frozenset({'39', '48'}), 0.0018440000000000002), (frozenset({'38', '48'}), 0.0018520000000000001), (frozenset({'32', '48'}), 0.000428), (frozenset({'522', '39'}), 4e-05), (frozenset({'522', '38'}), 4e-05), (frozenset({'249', '41'}), 4e-05), (frozenset({'718', '39'}), 2.8e-05), (frozenset({'718', '38'}), 2.8e-05), (frozenset({'2990', '48'}), 2.8e-05), (frozenset({'38', '94'}), 2.4e-05), (frozenset({'39', '94'}), 2.4e-05), (frozenset({'48', '1831'}), 3.5999999999999994e-05), (frozenset({'751', '48'}), 2.8e-05), (frozenset({'38', '51'}), 3.2e-05), (frozenset({'39', '51'}), 3.2e-05), (frozenset({'39', '1188'}), 4e-05), (frozenset({'39', '156'}), 4e-05), (frozenset({'38', '156'}), 4e-05), (frozenset({'38', '1188'}), 4e-05), (frozenset({'48', '352'}), 4.8e-05), (frozenset({'101', '48'}), 4.8e-05), (frozenset({'38', '237'}), 0.000104), (frozenset({'39', '237'}), 0.000104), (frozenset({'39', '1121'}), 8e-05), (frozenset({'38', '1121'}), 8e-05), (frozenset({'2694', '48'}), 2.4e-05), (frozenset({'38', '2673'}), 2.4e-05), (frozenset({'39', '2673'}), 2e-05), (frozenset({'37', '41'}), 8.400000000000001e-05), (frozenset({'48', '89'}), 0.000136), (frozenset({'2958', '48'}), 3.5999999999999994e-05), (frozenset({'438', '41'}), 2.4e-05), (frozenset({'48', '267'}), 2.4e-05), (frozenset({'38', '3270'}), 4.4e-05), (frozenset({'48', '749'}), 2.4e-05), (frozenset({'39', '3270'}), 4.4e-05), (frozenset({'56', '38'}), 4.8e-05), (frozenset({'56', '39'}), 4.8e-05), (frozenset({'189', '48'}), 4.4e-05), (frozenset({'39', '384'}), 3.5999999999999994e-05), (frozenset({'371', '41'}), 3.2e-05), (frozenset({'32', '1121'}), 2.4e-05), (frozenset({'179', '48'}), 5.6e-05), (frozenset({'39', '225'}), 0.000136), (frozenset({'225', '32'}), 4.8e-05), (frozenset({'38', '49'}), 4.4e-05), (frozenset({'123', '41'}), 2e-05), (frozenset({'39', '49'}), 4.4e-05), (frozenset({'32', '49'}), 2.4e-05), (frozenset({'38', '225'}), 0.000136), (frozenset({'185', '48'}), 2.4e-05), (frozenset({'39', '1146'}), 7.199999999999999e-05), (frozenset({'38', '1146'}), 7.199999999999999e-05), (frozenset({'740', '39'}), 5.2e-05), (frozenset({'740', '38'}), 5.2e-05), (frozenset({'38', '1715'}), 7.199999999999999e-05), (frozenset({'39', '1715'}), 7.199999999999999e-05), (frozenset({'32', '1715'}), 2.8e-05), (frozenset({'55', '48'}), 5.2e-05), (frozenset({'68', '48'}), 2.8e-05), (frozenset({'32', '1146'}), 3.2e-05), (frozenset({'727', '41'}), 2e-05), (frozenset({'89', '1121'}), 2e-05), (frozenset({'441', '48'}), 2.4e-05), (frozenset({'757', '39'}), 2.8e-05), (frozenset({'757', '38'}), 2.8e-05), (frozenset({'161', '39'}), 2.4e-05), (frozenset({'161', '38'}), 2.4e-05), (frozenset({'370', '41'}), 3.5999999999999994e-05), (frozenset({'1327', '48'}), 8.400000000000001e-05), (frozenset({'48', '78'}), 2.8e-05), (frozenset({'357', '41'}), 2e-05), (frozenset({'271', '41'}), 2.4e-05), (frozenset({'38', '384'}), 3.2e-05), (frozenset({'39', '170'}), 0.000368), (frozenset({'38', '170'}), 0.000368), (frozenset({'105', '48'}), 7.6e-05), (frozenset({'281', '48'}), 8.400000000000001e-05), (frozenset({'39', '681'}), 2.4e-05), (frozenset({'38', '110'}), 0.000328), (frozenset({'38', '681'}), 2.4e-05), (frozenset({'110', '32'}), 7.199999999999999e-05), (frozenset({'39', '110'}), 0.000328), (frozenset({'2505', '38'}), 3.2e-05), (frozenset({'310', '36'}), 2.4e-05), (frozenset({'310', '255'}), 2e-05), (frozenset({'2505', '39'}), 3.2e-05), (frozenset({'381', '39'}), 2.4e-05), (frozenset({'381', '38'}), 2.4e-05), (frozenset({'170', '32'}), 5.9999999999999995e-05), (frozenset({'170', '89'}), 2.4e-05), (frozenset({'1327', '110'}), 2e-05), (frozenset({'1859', '39'}), 3.2e-05), (frozenset({'1859', '38'}), 2.8e-05), (frozense

In [5]:
input_rdd = sc.textFile('/FileStore/tables/num_1000.txt',8)

In [6]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/num_1000.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import sys
FreqSet-2:
seconds process time: 1.1965950000000003
seconds wall time: 2.3083183765411377
[(frozenset({'55', '48'}), 3.5000000000000004e-05), (frozenset({'32', '48'}), 0.001), (frozenset({'39', '48'}), 0.001), (frozenset({'38', '48'}), 0.001), (frozenset({'151', '32'}), 8.999999999999999e-06), (frozenset({'2363', '48'}), 6e-06), (frozenset({'151', '38'}), 8.999999999999999e-06), (frozenset({'151', '39'}), 8.999999999999999e-06), (frozenset({'107', '38'}), 2e-05), (frozenset({'107', '32'}), 2e-05), (frozenset({'32', '662'}), 8e-06), (frozenset({'38', '662'}), 8e-06), (frozenset({'107', '39'}), 2e-05), (frozenset({'39', '662'}), 8e-06), (frozenset({'56', '38'}), 3.5999999999999994e-05), (frozenset({'1090', '48'}), 8e-06), (frozenset({'56', '32'}), 3.5999999999999994e-05), (frozenset({'56', '39'}), 3.5999999999999994e-05), (frozenset({'101', '48'}), 2.7e-05), (frozenset({'32', '1004'}), 2e-05), (frozenset({'38', '488'}), 8e-06), (frozenset({'38', '1004'}), 2e-05), (frozenset({'2775', '48'}), 8.999999999999999e-06), (frozenset({'39', '488'}), 8e-06), (frozenset({'39', '1004'}), 2e-05), (frozenset({'488', '32'}), 8e-06), (frozenset({'48', '425'}), 6e-06), (frozenset({'1144', '38'}), 7e-06), (frozenset({'1144', '39'}), 7e-06), (frozenset({'32', '49'}), 2.8e-05), (frozenset({'1144', '32'}), 7e-06), (frozenset({'39', '49'}), 2.8e-05), (frozenset({'38', '49'}), 2.8e-05), (frozenset({'39', '3617'}), 1.3e-05), (frozenset({'38', '3617'}), 1.3e-05), (frozenset({'3617', '32'}), 1.3e-05), (frozenset({'3617', '2378'}), 6e-06), (frozenset({'48', '2378'}), 2.7e-05), (frozenset({'237', '32'}), 6.500000000000001e-05), (frozenset({'237', '89'}), 1e-05), (frozenset({'48', '89'}), 8.400000000000001e-05), (frozenset({'38', '237'}), 6.500000000000001e-05), (frozenset({'39', '237'}), 6.500000000000001e-05), (frozenset({'5124', '48'}), 6e-06), (frozenset({'38', '10451'}), 6e-06), (frozenset({'10451', '32'}), 6e-06), (frozenset({'7932', '48'}), 7e-06), (frozenset({'39', '10451'}), 6e-06), (frozenset({'31', '39'}), 1.2e-05), (frozenset({'31', '32'}), 1.2e-05), (frozenset({'31', '38'}), 1.2e-05), (frozenset({'1652', '48'}), 7e-06), (frozenset({'1', '48'}), 8e-06), (frozenset({'790', '956'}), 5e-06), (frozenset({'48', '78'}), 2.6e-05), (frozenset({'1727', '48'}), 1.4999999999999999e-05), (frozenset({'38', '1062'}), 6e-06), (frozenset({'39', '1062'}), 6e-06), (frozenset({'32', '1062'}), 6e-06), (frozenset({'597', '48'}), 6e-06), (frozenset({'805', '48'}), 1.1e-05), (frozenset({'961', '48'}), 1.1e-05), (frozenset({'23', '38'}), 1.2e-05), (frozenset({'23', '32'}), 1.2e-05), (frozenset({'23', '39'}), 1.2e-05), (frozenset({'23', '89'}), 5e-06), (frozenset({'2958', '48'}), 1.7999999999999997e-05), (frozenset({'38', '1986'}), 1.7999999999999997e-05), (frozenset({'39', '1986'}), 1.7999999999999997e-05), (frozenset({'1986', '32'}), 1.7999999999999997e-05), (frozenset({'164', '32'}), 8e-06), (frozenset({'164', '38'}), 8e-06), (frozenset({'165', '38'}), 7e-06), (frozenset({'164', '39'}), 8e-06), (frozenset({'165', '32'}), 7e-06), (frozenset({'165', '39'}), 7e-06), (frozenset({'150', '48'}), 8.999999999999999e-06), (frozenset({'168', '38'}), 1e-05), (frozenset({'168', '39'}), 1e-05), (frozenset({'168', '32'}), 1e-05), (frozenset({'39', '225'}), 7.7e-05), (frozenset({'38', '225'}), 7.7e-05), (frozenset({'225', '32'}), 7.7e-05), (frozenset({'237', '805'}), 5e-06), (frozenset({'48', '1831'}), 6e-06), (frozenset({'341', '32'}), 7e-06), (frozenset({'341', '38'}), 7e-06), (frozenset({'39', '3270'}), 1.9e-05), (frozenset({'341', '39'}), 7e-06), (frozenset({'3270', '32'}), 1.9e-05), (frozenset({'38', '3270'}), 1.9e-05), (frozenset({'185', '48'}), 2e-05), (frozenset({'39', '186'}), 1.6e-05), (frozenset({'38', '186'}), 1.6e-05), (frozenset({'32', '186'}), 1.6e-05), 

In [7]:
input_rdd = sc.textFile('/FileStore/tables/num_10000.txt',8)

In [8]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/num_10000.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import sys
FreqSet-2:
seconds process time: 0.5403400000000005
seconds wall time: 1.3377034664154053
[(frozenset({'237', '32'}), 1.03e-06), (frozenset({'39', '48'}), 8.958e-05), (frozenset({'38', '48'}), 5.381e-05), (frozenset({'32', '48'}), 1.965e-05), (frozenset({'48', '89'}), 6.469999999999999e-06), (frozenset({'38', '237'}), 3.37e-06), (frozenset({'39', '237'}), 4.66e-06), (frozenset({'2775', '48'}), 5.3e-07), (frozenset({'107', '39'}), 1.38e-06), (frozenset({'107', '38'}), 8.3e-07), (frozenset({'39', '156'}), 1.31e-06), (frozenset({'38', '156'}), 1e-06), (frozenset({'1257', '48'}), 5.8e-07), (frozenset({'39', '1188'}), 8.8e-07), (frozenset({'179', '48'}), 1.55e-06), (frozenset({'38', '1188'}), 9.299999999999999e-07), (frozenset({'39', '10442'}), 7e-07), (frozenset({'31', '39'}), 1.19e-06), (frozenset({'805', '48'}), 5.3e-07), (frozenset({'31', '38'}), 7.7e-07), (frozenset({'39', '225'}), 6.169999999999999e-06), (frozenset({'38', '225'}), 3.73e-06), (frozenset({'38', '1146'}), 1.62e-06), (frozenset({'39', '1146'}), 3.1600000000000002e-06), (frozenset({'32', '1146'}), 8.1e-07), (frozenset({'101', '48'}), 2.83e-06), (frozenset({'39', '80'}), 7.7e-07), (frozenset({'39', '984'}), 7e-07), (frozenset({'808', '48'}), 5.3e-07), (frozenset({'264', '48'}), 1.76e-06), (frozenset({'39', '1020'}), 5.3e-07), (frozenset({'55', '48'}), 2.16e-06), (frozenset({'23', '39'}), 1.04e-06), (frozenset({'225', '32'}), 1.5e-06), (frozenset({'48', '1831'}), 6.8e-07), (frozenset({'48', '78'}), 1.8599999999999998e-06), (frozenset({'39', '1062'}), 5.9e-07), (frozenset({'39', '715'}), 5.499999999999999e-07), (frozenset({'168', '39'}), 6e-07), (frozenset({'38', '389'}), 9.6e-07), (frozenset({'39', '389'}), 1.48e-06), (frozenset({'209', '39'}), 6.100000000000001e-07), (frozenset({'39', '1003'}), 5.6e-07), (frozenset({'39', '1693'}), 6.3e-07), (frozenset({'161', '39'}), 1.3899999999999998e-06), (frozenset({'161', '38'}), 8.5e-07), (frozenset({'56', '38'}), 2.09e-06), (frozenset({'56', '39'}), 2.0199999999999997e-06), (frozenset({'225', '89'}), 5.2e-07), (frozenset({'56', '32'}), 5e-07), (frozenset({'39', '49'}), 1.8599999999999998e-06), (frozenset({'1196', '48'}), 6.3e-07), (frozenset({'38', '49'}), 1.24e-06), (frozenset({'32', '49'}), 5e-07), (frozenset({'39', '2673'}), 6.3e-07), (frozenset({'1144', '38'}), 5.3e-07), (frozenset({'1144', '39'}), 8.8e-07), (frozenset({'1052', '39'}), 6.2e-07), (frozenset({'39', '488'}), 1.09e-06), (frozenset({'38', '488'}), 6.100000000000001e-07), (frozenset({'48', '425'}), 8e-07), (frozenset({'1727', '48'}), 5.6e-07), (frozenset({'48', '267'}), 1.07e-06), (frozenset({'910', '48'}), 8.9e-07), (frozenset({'2056', '39'}), 6.499999999999999e-07), (frozenset({'1239', '48'}), 7.6e-07), (frozenset({'740', '39'}), 1.25e-06), (frozenset({'740', '38'}), 8.200000000000001e-07), (frozenset({'718', '39'}), 6.6e-07), (frozenset({'48', '675'}), 1.04e-06), (frozenset({'319', '48'}), 6e-07), (frozenset({'150', '48'}), 7.5e-07), (frozenset({'185', '48'}), 2.11e-06), (frozenset({'522', '38'}), 6.9e-07), (frozenset({'1355', '48'}), 8.399999999999999e-07), (frozenset({'522', '39'}), 1.31e-06), (frozenset({'2958', '48'}), 1.6400000000000002e-06), (frozenset({'39', '3270'}), 1.55e-06), (frozenset({'38', '3270'}), 1.03e-06), (frozenset({'38', '2241'}), 5.8e-07), (frozenset({'39', '2241'}), 8.5e-07), (frozenset({'961', '48'}), 7.7e-07), (frozenset({'341', '39'}), 5.4e-07), (frozenset({'396', '48'}), 1.03e-06), (frozenset({'39', '1209'}), 6.3e-07), (frozenset({'39', '1976'}), 5.499999999999999e-07), (frozenset({'19', '48'}), 1.25e-06), (frozenset({'1404', '48'}), 6.3e-07), (frozenset({'1327', '48'}), 3.4299999999999998e-06), (frozenset({'39', '1004'}), 1.18e-06), (frozenset({'38', '1004'}), 7.2e-07), (frozenset({'39', '186'}), 9.5e-07), (frozense

In [9]:
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)

In [10]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import sys
FreqSet-2:
seconds process time: 0.20290299999999206
seconds wall time: 1.050638198852539
[(frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'19', '48'}), 7.60369368984735e-08), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'45', '39'}), 7.191987770938525e-08), (frozenset({'39', '242'}), 7.256316820768029e-08), (frozenset({'258', '48'}), 8.27271580807419e-08), (frozenset({'286', '38'}), 1.435824392194525e-07), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'11', '48'}), 5.956870014212052e-08), (frozenset({'405', '39'}), 6.432904982950381e-08), (frozenset({'48', '413'}), 1.4602694311297362e-07), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'270', '41'}), 6.535831462677586e-08), (frozenset

In [11]:
#=============================================================================="修改partition=4"==================================="

In [12]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改partition=4"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',4)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.18975400000000775
seconds wall time: 1.3882465362548828
[(frozenset({'589', '48'}), 8.079728658585678e-08), (frozenset({'338', '48'}), 1.0022465963436694e-07), (frozenset({'39', '269'}), 5.673822194962236e-08), (frozenset({'39', '105'}), 5.853943534484846e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'170', '48'}), 2.0032066116907482e-07), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'38', '105'}), 8.27271580807419e-08), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'677', '48'}), 8.568629437289907e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'19', '48'}), 7.60369368984735e-08), (frozenset({'1814', '39'}), 6.059796493939259e-08), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'548', '48'}), 8.465702957562701e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'110', '48'}), 1.775481775294305e-07), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'201', '48'}), 8.671555917017112e-08), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'48', '76'}), 6.23991783346187e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'39', '533'}), 1.18622767885605e-07), (frozenset({'703', '48'}), 6.008333254075655e-08), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '117'}), 7.127658721109021e-08), (frozenset({'39', '976'}), 6.059796493939259e-08), (frozenset({'39', '812'}), 6.059796493939259e-08), (frozenset({'270', '39'}), 1.536177709928551e-07), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'766', '48'}), 6.291381073325472e-08), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'479', '48'}), 6.600160512507091e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'334', '39'}), 6.510099842745785e-08), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'39', '101'}), 1.8012133952261066e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'38', '39'}), 1.3309680409724336e-06), (frozenset({'38', '32'}), 3.644883963339686e-07), (frozenset({'31', '48'}), 6.342844313189076e-08), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'237', '48'}), 2.164029236264508e-07), (frozenset({'225', '48'}), 2.2335046100803718e-07), (frozenset({'1004', '48'}), 7.899607319063067e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'48', '156'}), 6.08552811387106e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'270

In [13]:
#=============================================================================="修改partition=8"==================================="


In [14]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改partition=8"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.18889999999998963
seconds wall time: 1.0639960765838623
[(frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'19', '48'}), 7.60369368984735e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'10515', '48'}), 6.741684422131999e-08), (frozenset({'48', '413'}), 1.4602694311297362e-07), (frozenset({'11', '48'}), 5.956870014212052e-08), (frozenset({'544', '39'}), 6.741684422131999e-08), (frozenset({'592', '48'}), 9.559296804664264e-08), (frozenset({'258', '48'}), 8.27271580807419e-08), (frozenset({'48', '649'}), 6.304246883291372e-08), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'270', '41'}), 6.535831462677586e-08), (froz

In [15]:
#=============================================================================="修改partition=12"==================================="

In [16]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改partition=12"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',12)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.24969000000000108
seconds wall time: 1.2778711318969727
[(frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'39', '105'}), 5.853943534484846e-08), (frozenset({'48', '175'}), 6.342844313189076e-08), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'39', '269'}), 5.673822194962236e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'110', '48'}), 1.775481775294305e-07), (frozenset({'201', '48'}), 8.671555917017112e-08), (frozenset({'39', '533'}), 1.18622767885605e-07), (frozenset({'48', '19'}), 7.60369368984735e-08), (frozenset({'479', '48'}), 6.600160512507091e-08), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'48', '76'}), 6.23991783346187e-08), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'334', '39'}), 6.510099842745785e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'39', '32'}), 1.0878042326169092e-06), (frozenset({'39', '101'}), 1.8012133952261066e-07), (frozenset({'38', '39'}), 1.3309680409724336e-06), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'185', '39'}), 1.0652890651765831e-07), (frozenset({'65', '41'}), 1.2801480916071258e-07), (frozenset({'38', '32'}), 3.644883963339686e-07), (frozenset({'740', '48'}), 6.690221182268396e-08), (frozenset({'48', '1146'}), 1.0434171882345518e-07), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'55', '38'}), 8.452837147596799e-08), (frozenset({'264', '39'}), 7.706620169574556e-08), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'39', '1435'}), 5.905406774348449e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'48', '156'}), 6.08552811387106e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'39', '175'}), 8.11832608848338e-08), (frozenset({'31', '48'}), 6.342844313189076e-08), (frozenset({'1344', '39'}), 6.23991783346187e-08), (frozenset({'65', '48'}), 3.2537633403763024e-07), (frozenset({'2238', '48'}), 1.2286848517435228e-07), (frozenset({'36', '48'}), 1.8217986911715478e-07), (frozenset({'37', '38'}), 1.3457637224332197e-07), (frozenset({'229', '48'}), 7.011866431415914e-08), (frozenset({'286', '38'}), 1.435824392194525e-07), (frozenset({'310', '48'}), 2.1768950462304087e-07), (frozenset({'48', '649'}), 6.304246883291372e-08), (frozenset({'1135', '48'}), 6.960403191552312e-08), (frozenset({'18', '48'}), 5.982601634143855e-08), (frozenset({'301', '48'}), 8.465702957562701e-08), (frozenset({'956', '39'}), 7.719485979540457e-08), (frozenset({'10515', '48'}), 6.741684422131999e-08), (frozenset({'1146', '41'}), 6.445770792916281e-08), (frozenset({'570', '39'}), 7.179121960972625e-08), (frozenset({'371', '39'}), 6.844610901859204e-08), (frozenset({'16010', '48'}), 9.392041275107556e-08), (frozenset({'39', '16217'}), 9.366309655175754e-08), (frozenset({'39', '41'}), 1.468503549507913e-06), (frozenset({'38', '41'}), 5.013806143711526e-07), (frozenset({'1578', '39'}), 7.410706540358838e-08), (frozenset({'39', '60'}), 1.2647091196480448e-07), (frozenset({'9', '39'}), 1.0704353891629432e-07), (frozenset({'39', '103'}), 6.58729470254119e-08), (frozenset({'270', '271'}), 8.748750776812517e-08), (frozenset({'48', '604'}), 8.838811446573822e-08), (frozenset({'570', '48'}), 6.613026322472991e-08), (frozenset({'37', '48'}), 7.269182630733929e-08), (frozenset({'772', '48'}), 5.6866880049281367e-08), (frozenset({

In [17]:
#=============================================================================="修改partition=16"==================================="


In [18]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改partition=16"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',16)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576150788356-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.2654759999999925
seconds wall time: 1.1900765895843506
[(frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'48', '19'}), 7.60369368984735e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'592', '48'}), 9.559296804664264e-08), (frozenset({'310', '48'}), 2.1768950462304087e-07), (frozenset({'286', '38'}), 1.435824392194525e-07), (frozenset({'39', '1393'}), 1.01511240630957e-07), (frozenset({'45', '39'}), 7.191987770938525e-08), (frozenset({'39', '604'}), 9.971002723573089e-08), (frozenset({'956', '39'}), 7.719485979540457e-08), (frozenset({'2238', '48'}), 1.2286848517435228e-07), (frozenset({'301', '48'}), 8.465702957562701e-08), (frozenset({'570', '39'}), 7.179121960972625e-08), (frozenset({'39', '13041'}), 8.851677256539724e-08), (frozenset({'16010', '48'}), 9.392041275107556e-08), (frozenset({'310', '39'}), 2.3827480056848208e-07), (frozenset({'301', '39'}), 9.199054125619043e-08), (frozenset({'956', '48'}), 6.947537381586411e-08), (frozenset({'48', '604'}), 8.838811446573822e-08), (frozenset({'310', '32'}), 6.960403191552312e-08), (frozenset({'592', '39'}), 9.301980605346249e-08), (frozenset({'48', '772'}), 5.6866880049281367e-08), (frozenset({'45', '48'}), 6.329978503223175e-08), (frozenset({'570', '48'}), 6.613026322472991e-08), (frozenset({'39', '2238'}), 1.655829742611428e-07), (frozenset({'1393', '48'}), 8.607226867187609e-08), (frozenset({'790', '38'}), 6.535831462677586e-08), (frozenset({'13041', '48'}), 7.745217599472258e-08), (frozenset({'16010', '39'}), 1.066575646173173e-07), (frozenset({'270', '39'}), 1.536177709928551e-07), (frozenset({'1814', '39'}), 6.059796493939259e-08), (frozenset({'39', '117'}), 7.127658721109021e-08), (frozenset({'548', '48'}), 8.465702957562701e-08), (frozenset({'110', '48'}), 1.775481775294305e-07), (frozenset({'38', '105'}), 8.27271580807419e-08), (frozenset({'766', '48'}), 6.291381073325472e-08), (frozenset({'338', '48'}), 1.0022465963436694e-07), (frozenset({'201', '48'}), 8.671555917017112e-08), (frozenset({'48', '76'}), 6.23991783346187e-08), (frozenset({'39', '812'}), 6.059796493939259e-08), (frozenset({'39', '533'}), 1.18622767885605e-07), (frozenset({'589', '48'}), 8.079728658585678e-08), (frozenset({'39', '976'}), 6.059796493939259e-08), (frozenset({'479', '48'}), 6.600160512507091e-08), (frozenset({'39', '269'}), 5.673822194962236e-08), (frozenset({'170', '48'}), 2.0032066116907482e-07), (frozenset({'161', '48'}), 6.651623752370693e-08), (frozenset({'39', '89'}), 3.536811159626119e-07), (frozenset({'39', '19'}), 7.61655949981325e-08), (frozens

In [19]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=2"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.21556299999999995
seconds wall time: 2.0589139461517334
[(frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'48', '19'}), 7.60369368984735e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'39', '783'}), 8.234118378176486e-08), (frozenset({'310', '48'}), 2.1768950462304087e-07), (frozenset({'956', '39'}), 7.719485979540457e-08), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'48', '649'}), 6.304246883291372e-08), (frozenset({'544', '39'}), 6.741684422131999e-08), (frozenset({'286', '38'}), 1.435824392194525e-07), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'1578', '48'}), 6.034064874007457e-08), (frozenset({'39', '604'}), 9.971002723573089e-08), (fro

In [20]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=3"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.1909679999999998
seconds wall time: 1.2658631801605225
[(frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'48', '19'}), 7.60369368984735e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'45', '39'}), 7.191987770938525e-08), (frozenset({'39', '242'}), 7.256316820768029e-08), (frozenset({'258', '48'}), 8.27271580807419e-08), (frozenset({'286', '38'}), 1.435824392194525e-07), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'11', '48'}), 5.956870014212052e-08), (frozenset({'405', '39'}), 6.432904982950381e-08), (frozenset({'48', '413'}), 1.4602694311297362e-07), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'270', '41'}), 6.535831462677586e-08), (frozense

In [21]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=4"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.26400999999999986
seconds wall time: 1.6436381340026855
[(frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'48', '19'}), 7.60369368984735e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'10515', '48'}), 6.741684422131999e-08), (frozenset({'48', '413'}), 1.4602694311297362e-07), (frozenset({'11', '48'}), 5.956870014212052e-08), (frozenset({'544', '39'}), 6.741684422131999e-08), (frozenset({'592', '48'}), 9.559296804664264e-08), (frozenset({'258', '48'}), 8.27271580807419e-08), (frozenset({'48', '649'}), 6.304246883291372e-08), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'270', '41'}), 6.535831462677586e-08), (froz

In [22]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=5"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




/local_disk0/tmp/1576167864364-0/PythonShell.py:13: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
 import time
FreqSet-2:
seconds process time: 0.20750499999999938
seconds wall time: 1.1835334300994873
[(frozenset({'32', '48'}), 1.0336391726604671e-06), (frozenset({'225', '32'}), 8.427105527664999e-08), (frozenset({'39', '48'}), 3.7493543402627996e-06), (frozenset({'56', '38'}), 6.613026322472991e-08), (frozenset({'39', '225'}), 3.024751922983269e-07), (frozenset({'39', '156'}), 7.487901400154242e-08), (frozenset({'38', '225'}), 8.761616586778418e-08), (frozenset({'39', '1146'}), 1.2647091196480448e-07), (frozenset({'38', '48'}), 1.0220599436911564e-06), (frozenset({'23', '39'}), 6.58729470254119e-08), (frozenset({'161', '39'}), 8.002533798790273e-08), (frozenset({'39', '237'}), 2.481814742422257e-07), (frozenset({'48', '78'}), 1.0601427411902228e-07), (frozenset({'1327', '48'}), 1.2454104046991936e-07), (frozenset({'10', '39'}), 5.815346104587144e-08), (frozenset({'264', '48'}), 6.870342521791006e-08), (frozenset({'48', '89'}), 3.5998536284590326e-07), (frozenset({'39', '389'}), 6.999000621450014e-08), (frozenset({'237', '32'}), 7.307780060631632e-08), (frozenset({'2958', '48'}), 1.0022465963436694e-07), (frozenset({'179', '48'}), 8.028265418722075e-08), (frozenset({'740', '39'}), 9.765149764118678e-08), (frozenset({'39', '3270'}), 7.410706540358838e-08), (frozenset({'185', '48'}), 1.0498500932175022e-07), (frozenset({'522', '39'}), 8.337044857903693e-08), (frozenset({'271', '41'}), 7.513633020086044e-08), (frozenset({'19', '48'}), 7.60369368984735e-08), (frozenset({'101', '48'}), 1.6867076865295898e-07), (frozenset({'38', '237'}), 8.78734820671022e-08), (frozenset({'31', '39'}), 6.92180576165461e-08), (frozenset({'39', '1004'}), 7.500767210120144e-08), (frozenset({'175', '48'}), 6.342844313189076e-08), (frozenset({'39', '806'}), 6.342844313189076e-08), (frozenset({'39', '49'}), 9.880942053811784e-08), (frozenset({'39', '110'}), 2.263095973001944e-07), (frozenset({'38', '110'}), 3.5059332157079575e-07), (frozenset({'334', '48'}), 5.699553814894037e-08), (frozenset({'39', '170'}), 2.6490702719789667e-07), (frozenset({'1814', '48'}), 5.802480294621243e-08), (frozenset({'38', '170'}), 3.8996270006645204e-07), (frozenset({'39', '589'}), 9.108993455857739e-08), (frozenset({'2238', '41'}), 7.333511680563434e-08), (frozenset({'533', '48'}), 1.1077462380640555e-07), (frozenset({'39', '789'}), 5.75101705475764e-08), (frozenset({'170', '32'}), 6.947537381586411e-08), (frozenset({'677', '39'}), 8.169789328346984e-08), (frozenset({'39', '479'}), 7.462169780222442e-08), (frozenset({'110', '32'}), 5.7767486746894416e-08), (frozenset({'39', '766'}), 6.046930683973357e-08), (frozenset({'812', '48'}), 5.956870014212052e-08), (frozenset({'39', '548'}), 8.530032007392205e-08), (frozenset({'39', '76'}), 6.806013471961502e-08), (frozenset({'270', '48'}), 1.2312580137367028e-07), (frozenset({'39', '338'}), 9.546430994698365e-08), (frozenset({'39', '201'}), 8.607226867187609e-08), (frozenset({'48', '117'}), 7.732351789506357e-08), (frozenset({'310', '41'}), 9.250517365482647e-08), (frozenset({'475', '41'}), 7.333511680563434e-08), (frozenset({'976', '48'}), 5.866809344450747e-08), (frozenset({'703', '39'}), 5.8282119145530445e-08), (frozenset({'39', '12929'}), 6.046930683973357e-08), (frozenset({'16010', '41'}), 6.008333254075655e-08), (frozenset({'10515', '48'}), 6.741684422131999e-08), (frozenset({'48', '413'}), 1.4602694311297362e-07), (frozenset({'11', '48'}), 5.956870014212052e-08), (frozenset({'544', '39'}), 6.741684422131999e-08), (frozenset({'592', '48'}), 9.559296804664264e-08), (frozenset({'258', '48'}), 8.27271580807419e-08), (frozenset({'48', '649'}), 6.304246883291372e-08), (frozenset({'286', '39'}), 9.649357474425571e-08), (frozenset({'475', '48'}), 1.8372376631306288e-07), (frozenset({'270', '41'}), 6.535831462677586e-08), (froz

In [23]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=6"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




In [24]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=7"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")




In [25]:
# coding=utf-8
#
# pyspark实现Apriori算法、循环迭代、并行处理
#encoding='utf-8'
#=============================================================================="修改w=8"==================================="
from pyspark import  SparkContext
sc = SparkContext.getOrCreate()

import pandas as pd
import numpy as np
import time

start = time.clock()
t0 = time.time()

#参数初始化
K = 200#迭代次数
support = 0.005
candidate_sets = []
current_set_let = 1

# 将输入数据分区，由于后面要频繁使用。因此缓存起来
input_rdd = sc.textFile('/FileStore/tables/retail.txt',8)
#input_rdd = sc.textFile('/FileStore/tables/gyy_test.txt',8)
#print(input_rdd.collect())



def splitlines(x):
    y = ' '.join(x.split())
    return y

splitedline = input_rdd.map(splitlines)
transations = splitedline.map(lambda x: x.split(' '))
transations.collect()
transations.cache()

numRecords = float(transations.count())
#print ("numRecords：" + str(numRecords))
freqThre = numRecords * support
#print ("frequency threshold：" + str(freqThre))

#计算频繁1项集
t1=time.clock()
t1_time=time.time()
candidate_1 =splitedline.flatMap(lambda x: x.split(' '))
FreqSet_1= candidate_1.map(lambda x: (x,1)).reduceByKey(lambda x, y: x + y)\
.filter(lambda x: x[1]>freqThre).map(lambda x: (set([x[0]]),x[1]/numRecords))
#print ('cur_freqset_1',FreqSet_1.collect())


preFreSets = FreqSet_1.map(lambda x: list(x[0])[0])
t1e=time.clock()
t1e_time=time.time()

def JUDGEt(t,elements):
    for ele in elements:
        s= list(t)
        if (ele not in list(t)):
            s.append(ele)                   
            yield (set(s))
    
def ProduceNminus1Subset(cset,cur_round,pre):
    N=cur_round
    tu = []
    csets=list(cset)
    N = len(csets)

    for j in range(N):
        combo = []
        flag = 'TRUE'
        for i in range(N):
            if(i!=j):
                combo.append(csets[i])
        
        combo2 = set(combo)
        flag = frozenset(combo2).issubset(pre)
        
        tu.append(combo)
        if flag == 'FALSE':
            return tu

    return tu
  
    
def verifyCandidates(transation, candidates) :
    for c in candidates :
        if (c.issubset(transation)):
             yield (c, 1)

frqtemp=[]                
for r in range(2,K):
    cyclestart=time.clock()
    cyclestart_time=time.time()
    #print ('round'+str(r)+':\n')
    #print (preFreSets.collect())
    if preFreSets.isEmpty():
        break
    
    pre = (set(preFreSets.collect()))
    #print ('pre:'+str(pre)+'\n')
    
    if r>2:
        elements = list(preFreSets.map(lambda x: set(x)).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set(x)).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))
        #print ('candidates in round'+str(r)+':\n'+str(candidates.collect())+'\n')
    
    else:
        elements = list(preFreSets.map(lambda x: set([x])).reduce(lambda a,b: a.union(b)))
        canSets = preFreSets.map(lambda x: set([x])).flatMap(lambda t: JUDGEt(t,elements))\
        .map(lambda x: frozenset(x)).distinct().map(lambda x: list(x))
        cantemp = canSets.map(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1]).collect()
        candidates = canSets.filter(lambda cset :ProduceNminus1Subset(cset,r,pre)[-1][-1])#
        candidates = candidates.map(lambda x: frozenset(x))

    
    
    broadcastCandidates = sc.broadcast(frozenset(list(candidates.collect())))
    curFreqSet = transations.map(lambda x: frozenset(x)).flatMap(lambda transation : verifyCandidates(transation, broadcastCandidates.value))\
                 .reduceByKey(lambda a,b: a+b)\
                 .map(lambda x: (x[0],x[1]/numRecords))\
                 .filter(lambda x :x[1] >= support)
    
    preFreSets = curFreqSet.map(lambda x :x[0])
    
    cycleend=time.clock()
    cycleend_time=time.time()
    ctemp = curFreqSet.map(lambda x: (x[0],x[1]/numRecords))
    if ctemp.count()>0:
      frqtemp.append("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    
    print("FreqSet-"+str(r)+':\n'+'seconds process time: '+str(cycleend-cyclestart)+'\n'+"seconds wall time: "+str(cycleend_time-cyclestart_time)+'\n'+str(ctemp.collect())+'\n')
    

  #写入频繁round项集结果到hdfs
with open("/dbfs/FileStore/tables/FreqSets.txt", 'w') as f:
  if FreqSet_1.count()>0:
    f.write("FreqSet-1"+':\n'+'seconds process time: '+str(t1e-t1)+'\n'+"seconds wall time: "+str(t1e_time-t1_time)+'\n'+str(FreqSet_1.collect())+'\n')    
  for frq in frqtemp:
    f.write(frq)
   

    
end = time.clock()
print (end - start, "seconds total process time")
print (time.time() - t0, "seconds total wall time")


